<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/09wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

{{<video https://youtu.be/playlist?list=PLQqh36zP38-xsRgsXpLEUNWClxLi0N9Mk&si=Lssxzw70RRT1adiJ >}}

In [1]:
!pip uninstall mp2024pkg -y 
!pip install git+https://github.com/guebin/mp2024pkg.git

Found existing installation: mp2024pkg 1.0
Uninstalling mp2024pkg-1.0:
  Successfully uninstalled mp2024pkg-1.0
  Cloning https://github.com/guebin/mp2024pkg.git to /tmp/pip-req-build-492c7x4k
  Running command git clone --filter=blob:none --quiet https://github.com/guebin/mp2024pkg.git /tmp/pip-req-build-492c7x4k
  Resolved https://github.com/guebin/mp2024pkg.git to commit 86f7741f20beb99dd57ed2b534528db30a70bc9a
  Preparing metadata (setup.py) ... done
  Created wheel for mp2024pkg: filename=mp2024pkg-1.0-py3-none-any.whl size=4451 sha256=b6b72746e5060e0eb30557ba6504f7373ca350a03fc67ce274b8ed116955fdf8
  Stored in directory: /tmp/pip-ephem-wheel-cache-uvqwgvtd/wheels/83/81/b1/63c03ca869bb1a35a588dce54e83ea32c6c21e6af0bbeaa640
Successfully built mp2024pkg


# 2. Imports 

In [2]:
import transformers
import datasets
import huggingface_hub
import torch
import torchvision
import pytorchvideo.data
import PIL
import tarfile
import mp2024pkg as mp

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. Model 입력파악

`-` 아래중 하나의 방법순으로.. 

- 방법1: `model.forward?` 에서 시그니처를 확인
- 방법2: `model.forward?` 에서 사용예제를 확인
- 방법3: 인터넷을 활용한 외부 자료 확인 (공식문서, 공식튜토리얼, 신뢰할만한 블로그, ChatGPT등)
- 방법4: `model.forward??` 를 보고 모든 코드를 뜯어봄 <--- 하지마세요

`-` 모델의 입력이 어떤형태로 정리되어야 하는지 알아내는 확실한 방법은 없음 

- 방법1,2,3 은 다른사람의 호의에 기대해야함.
- 방법4는 사실상 불가능

`# 예제1` -- 텍스트분류

In [3]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [4]:
model1.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

- max_position_embeddings: 512

*모델의 입력파악*

In [5]:
model1.forward?

Signature:
model1.forward(
    input_ids: Optional[torch.Tensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    labels: Optional[torch.LongTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[transformers.modeling_outputs.SequenceClassifierOutput, Tuple[torch.Tensor, ...]]
Docstring:
The [`DistilBertForSequenceClassification`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
        Indices of input sequence 

*사용예시1 -- 입력나열, loss O*

In [6]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)

SequenceClassifierOutput(loss=tensor(0.6902, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0566, -0.0616],
        [-0.0505, -0.0575]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [7]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]]),
    labels = torch.tensor([0,0])
)
model1(**model1_input)

SequenceClassifierOutput(loss=tensor(0.6902, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0566, -0.0616],
        [-0.0505, -0.0575]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [8]:
model1(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0566, -0.0616],
        [-0.0505, -0.0575]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [9]:
model1_input = dict(
    input_ids = torch.tensor([[1,2,3,4], [2,3,4,5]])
)
model1(**model1_input)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0566, -0.0616],
        [-0.0505, -0.0575]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [10]:
model1(torch.tensor([[1,2,3,4], [2,3,4,5]]))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0566, -0.0616],
        [-0.0505, -0.0575]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

> 사용예시1~5에서 `model1()` 대신에 `model1.forward()`를 사용해도 된다. 

`#`

`# 예제2` -- 이미지분류

In [11]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [12]:
model2.config

ViTConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.2"
}

- image_size: 224

In [13]:
mp.tab(model2)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [14]:
model2.config.num_channels

3

*모델의 입력파악*

In [15]:
torch.randn(2,3,64,64)

tensor([[[[-0.1009,  0.0782, -0.2936,  ..., -0.8261, -1.1317,  0.9066],
          [ 0.7733,  1.4631, -0.6633,  ..., -0.3680,  0.2449, -0.7789],
          [-1.2995, -1.8838,  0.1308,  ..., -0.2662, -0.4356,  0.8953],
          ...,
          [-1.6264, -1.3875,  0.8295,  ...,  0.4387,  2.0692, -0.3382],
          [ 0.8416, -0.3607,  0.9601,  ...,  0.6325, -0.2735,  0.5620],
          [ 2.3128,  0.3432,  0.4706,  ..., -1.4477, -0.5906,  1.6101]],

         [[-0.7485,  0.7549, -1.2641,  ..., -0.7279,  0.0960, -0.4647],
          [-0.8664, -0.3963, -0.5144,  ..., -0.1854, -0.0972,  0.2777],
          [-0.2527,  0.0363,  1.7765,  ...,  2.2673, -0.6216,  0.0126],
          ...,
          [ 0.1170,  0.1373,  1.7322,  ...,  1.0348, -0.0675,  1.2776],
          [ 0.7451,  2.3603,  0.2400,  ...,  2.9033, -0.1395, -0.4768],
          [-1.1253,  1.3700,  0.2835,  ..., -0.1808,  0.2924,  0.3762]],

         [[ 1.6625,  0.2998,  0.2963,  ..., -0.1092, -0.1848, -1.0980],
          [-1.5162,  0.6258,  

*사용예시1 -- 입력나열, loss O*

In [16]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)

ImageClassifierOutput(loss=tensor(1.1295, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0349, -0.0951,  0.0233],
        [ 0.0300, -0.1247,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [17]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
    labels = torch.tensor([0,1])
)
model2(**model2_input)

ImageClassifierOutput(loss=tensor(1.1295, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0349, -0.0951,  0.0233],
        [ 0.0300, -0.1247,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [18]:
torch.random.manual_seed(42)
model2(
    pixel_values = torch.randn(2,3,224,224)
)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0349, -0.0951,  0.0233],
        [ 0.0300, -0.1247,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [19]:
torch.random.manual_seed(42)
model2_input = dict(
    pixel_values = torch.randn(2,3,224,224),
)
model2(**model2_input)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0349, -0.0951,  0.0233],
        [ 0.0300, -0.1247,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [20]:
torch.random.manual_seed(42)
model2(torch.randn(2,3,224,224))

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0349, -0.0951,  0.0233],
        [ 0.0300, -0.1247,  0.0345]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제3` -- 동영상분류

In [21]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*모델의 기본정보(config)*

In [22]:
model3.config

VideoMAEConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "MCG-NJU/videomae-base",
  "architectures": [
    "VideoMAEForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 384,
  "decoder_intermediate_size": 1536,
  "decoder_num_attention_heads": 6,
  "decoder_num_hidden_layers": 4,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "videomae",
  "norm_pix_loss": true,
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 16,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.2",
  "tubelet_size": 2,
  "use_mean_pooling": false
}

- image_size: 224
- num_frames: 16

*모델의 입력파악*

In [23]:
model3.forward?

Signature:
model3.forward(
    pixel_values: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    labels: Optional[torch.Tensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple, transformers.modeling_outputs.ImageClassifierOutput]
Docstring:
The [`VideoMAEForVideoClassification`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    pixel_values (`torch.FloatTensor` of shape `(batch_size, num_frames, num_channels, height, width)`):
        Pixel values. Pixel values can be obtained using [`AutoImageProcessor`]. See
        [`VideoMAEImageProcessor.__call__`] for de

*사용예시1 -- 입력나열, loss O*

In [24]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)

ImageClassifierOutput(loss=tensor(0.7254, grad_fn=<NllLossBackward0>), logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시2 -- `**딕셔너리`, loss O*

In [25]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
    labels = torch.tensor([0,1,0,1])
)
model3(**model3_input)

ImageClassifierOutput(loss=tensor(0.7254, grad_fn=<NllLossBackward0>), logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시3 -- 입력나열, loss X*

In [26]:
torch.random.manual_seed(42)
model3(
    pixel_values = torch.randn(4,16,3,224,224)
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시4 -- `**딕셔너리`, loss X*

In [27]:
torch.random.manual_seed(42)
model3_input = dict(
    pixel_values = torch.randn(4,16,3,224,224),
)
model3(**model3_input)

ImageClassifierOutput(loss=None, logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

*사용예시5 -- 초간단, loss X*

In [28]:
torch.random.manual_seed(42)
model3(torch.randn(4,16,3,224,224))

ImageClassifierOutput(loss=None, logits=tensor([[-0.7084, -0.2919],
        [-0.5846, -0.1854],
        [-0.6635, -0.2402],
        [-0.7193, -0.3811]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

# 4. Model 사용 연습 

## A. 텍스트

In [29]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- imdb

In [30]:
imdb = datasets.load_dataset('imdb')

In [31]:
d = imdb['train'].select(range(3))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

`(풀이1)`

*실패*

In [32]:
model1.forward(torch.tensor(tokenizer(d['text'])['input_ids']))

ValueError: expected sequence of length 363 at dim 1 (got 304)

*원인분석*

In [33]:
mp.show_list(
    tokenizer(d['text'])['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 363
   - Value: [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 205

In [34]:
mp.show_list(
    tokenizer(d['text'], padding=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 363
   - Value: [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 205

*성공*

In [35]:
model1(torch.tensor(tokenizer(d['text'], padding=True)['input_ids']))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=None, logits=tensor([[-0.0174,  0.1774],
        [-0.0269,  0.2127],
        [-0.0322,  0.2528]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`(풀이2)`

In [36]:
model1(tokenizer(d['text'], padding=True, return_tensors="pt")['input_ids'])

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0174,  0.1774],
        [-0.0269,  0.2127],
        [-0.0322,  0.2528]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제2` -- emotion

In [37]:
emotion = datasets.load_dataset('emotion')

In [38]:
d = emotion['train'].select(range(3))
d

Dataset({
    features: ['text', 'label'],
    num_rows: 3
})

`(풀이)`

In [39]:
model1(torch.tensor(tokenizer(d['text'],padding=True)['input_ids']))

SequenceClassifierOutput(loss=None, logits=tensor([[0.0163, 0.2285],
        [0.0100, 0.2066],
        [0.0087, 0.2619]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제3` -- MBTI

In [72]:
# mbti1.csv 파일 다운로드 
!wget https://raw.githubusercontent.com/guebin/MP2024/refs/heads/main/posts/mbti_1.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-11-12 22:32:08--  https://raw.githubusercontent.com/guebin/MP2024/refs/heads/main/posts/mbti_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62856486 (60M) [text/plain]
Saving to: ‘mbti_1.csv’

mbti_1.csv          100%[===================>]  59.94M   109MB/s    in 0.6s    

2024-11-12 22:32:09 (109 MB/s) - ‘mbti_1.csv’ saved [62856486/62856486]



In [44]:
d = datasets.Dataset.from_csv("mbti_1.csv").select(range(3))
d

Dataset({
    features: ['type', 'posts'],
    num_rows: 3
})

`(풀이1)`

*실패*

In [45]:
model1(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

Token indices sequence length is longer than the specified maximum sequence length for this model (2102 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (2102) must match the size of tensor b (512) at non-singleton dimension 1

*원인분석*

In [46]:
mp.show_list(
    tokenizer(d['posts'],padding=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 2102
   - Value: [101, 1005, 8299, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1053, 2015, 2595, 16257, 8545, 2509, 21638, 2860, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 4601, 1012, 2865, 1012, 10722, 14905, 20974, 1012, 4012, 1013, 10722, 14905, 20974, 1035, 1048, 14876, 26230, 2692, 2509, 9737, 27717, 19062, 2487, 3217, 9541, 2487, 1035, 3156, 1012, 16545, 2290, 1064, 1064, 1064, 4372, 22540, 1998, 20014, 3501, 5312, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1045, 2480, 2581, 2571, 2487, 2290, 2549, 2595, 2213, 2549, 2998, 13013, 2121, 2025, 2327, 2702, 3248, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 15384, 20952, 4371, 2487, 12870, 2278, 26418, 2015, 1064, 1064, 1064, 2054, 2038, 2042, 1996, 2087, 2166, 1011, 5278, 3325, 1999, 2115, 2166, 1029, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 7479, 10

In [47]:
mp.show_list(
    tokenizer(d['posts'],truncation=True)['input_ids']
)

List Overview:
Total items: 3

1. list[0]
   - Type: list
   - Length: 512
   - Value: [101, 1005, 8299, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1053, 2015, 2595, 16257, 8545, 2509, 21638, 2860, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 4601, 1012, 2865, 1012, 10722, 14905, 20974, 1012, 4012, 1013, 10722, 14905, 20974, 1035, 1048, 14876, 26230, 2692, 2509, 9737, 27717, 19062, 2487, 3217, 9541, 2487, 1035, 3156, 1012, 16545, 2290, 1064, 1064, 1064, 4372, 22540, 1998, 20014, 3501, 5312, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 1045, 2480, 2581, 2571, 2487, 2290, 2549, 2595, 2213, 2549, 2998, 13013, 2121, 2025, 2327, 2702, 3248, 16770, 1024, 1013, 1013, 7479, 1012, 7858, 1012, 4012, 1013, 3422, 1029, 1058, 1027, 15384, 20952, 4371, 2487, 12870, 2278, 26418, 2015, 1064, 1064, 1064, 2054, 2038, 2042, 1996, 2087, 2166, 1011, 5278, 3325, 1999, 2115, 2166, 1029, 1064, 1064, 1064, 8299, 1024, 1013, 1013, 7479, 101

*성공*

In [48]:
model1(torch.tensor(tokenizer(d['posts'],truncation=True)['input_ids']))
#model1(tokenizer(d['posts'],truncation=True,return_tensors="pt")['input_ids'])

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0450,  0.1728],
        [-0.0259,  0.1809],
        [-0.0611,  0.1994]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`(풀이2)` *--모델설정변경 (퀴즈5, 모델의 프레임수를 4로 바꾸는 예제에서 사용한 테크닉)*

*distilbert/distilbert-base-uncased 설정값 부르기*

In [49]:
config = transformers.AutoConfig.from_pretrained(
    "distilbert/distilbert-base-uncased"
)
config

DistilBertConfig {
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

*설정값변경*

In [50]:
config.max_position_embeddings = 2200

*설정값으로 모델불러오기*

In [51]:
model1_large = transformers.AutoModelForSequenceClassification.from_config(
    config=config
)

*모델사용*

In [52]:
model1_large(torch.tensor(tokenizer(d['posts'],padding=True)['input_ids']))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1023,  0.1124],
        [ 0.1443, -0.0719],
        [ 0.2653, -0.0368]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [53]:
model1_large(**tokenizer(d['posts'],padding=True,return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[0.1730, 0.0042],
        [0.2185, 0.0896],
        [0.3406, 0.0155]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제4` -- sms_spam

In [54]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})

In [55]:
d = sms_spam['train'].select(range(3))
d

Dataset({
    features: ['sms', 'label'],
    num_rows: 3
})

`(풀이)`

In [56]:
model1(**tokenizer(d['sms'],padding=True,return_tensors="pt"))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0118,  0.1699],
        [ 0.0273,  0.1815],
        [-0.0208,  0.2202]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

## B. 이미지

In [57]:
model2 = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=3 # 그냥 대충 3이라고 했음.. 별 이유는 없음
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- food101

In [58]:
d = datasets.load_dataset("food101", split="train[:4]")
d

Dataset({
    features: ['image', 'label'],
    num_rows: 4
})

`(예비학습)` -- `torchvision.transforms` 에서 제공하는 기능들은 배치처리가 가능한가? 

In [59]:
to_tensor = torchvision.transforms.ToTensor()

In [60]:
to_tensor(d['image'][0])

tensor([[[0.1216, 0.1137, 0.1098,  ..., 0.0039, 0.0039, 0.0000],
         [0.1255, 0.1216, 0.1176,  ..., 0.0039, 0.0039, 0.0000],
         [0.1294, 0.1255, 0.1255,  ..., 0.0039, 0.0000, 0.0000],
         ...,
         [0.2588, 0.2745, 0.2863,  ..., 0.3765, 0.3882, 0.3922],
         [0.2353, 0.2471, 0.2667,  ..., 0.3373, 0.3373, 0.3373],
         [0.2235, 0.2275, 0.2471,  ..., 0.3333, 0.3176, 0.3059]],

        [[0.1373, 0.1294, 0.1255,  ..., 0.1020, 0.1020, 0.0980],
         [0.1412, 0.1373, 0.1333,  ..., 0.1020, 0.1020, 0.0980],
         [0.1451, 0.1412, 0.1412,  ..., 0.1020, 0.0980, 0.0980],
         ...,
         [0.2471, 0.2627, 0.2745,  ..., 0.3647, 0.3765, 0.3882],
         [0.2235, 0.2353, 0.2549,  ..., 0.3255, 0.3333, 0.3333],
         [0.2118, 0.2157, 0.2353,  ..., 0.3216, 0.3137, 0.3020]],

        [[0.1412, 0.1333, 0.1294,  ..., 0.0902, 0.0902, 0.0863],
         [0.1451, 0.1412, 0.1451,  ..., 0.0902, 0.0902, 0.0863],
         [0.1490, 0.1451, 0.1529,  ..., 0.0902, 0.0863, 0.

In [61]:
to_tensor(d['image'])

TypeError: pic should be PIL Image or ndarray. Got <class 'list'>

`(풀이)`

In [62]:
compose = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224,224))
])

In [63]:
torch.stack(list(map(compose,d['image'])),axis=0).shape

torch.Size([4, 3, 224, 224])

In [64]:
model2.forward(
    torch.stack(list(map(compose,d['image'])),axis=0)
)

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0472,  0.0317, -0.1932],
        [ 0.2093,  0.1209, -0.0129],
        [ 0.0555,  0.0957, -0.0811],
        [-0.0682,  0.0363,  0.0317]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

`# 예제2`

In [65]:
beans = datasets.load_dataset('beans')
d = beans['train'].select(range(4))
d

Dataset({
    features: ['image_file_path', 'image', 'labels'],
    num_rows: 4
})

`(풀이)`

In [66]:
model2(torch.stack(list(map(compose,d['image'])),axis=0))

ImageClassifierOutput(loss=None, logits=tensor([[ 0.0545, -0.0993, -0.0386],
        [ 0.0576, -0.1576, -0.0019],
        [ 0.0982, -0.1531, -0.1147],
        [ 0.0570, -0.0575, -0.1455]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

`#`

## C. 동영상 

In [67]:
model3 = transformers.VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


`# 예제1` -- UCF101_subset

In [68]:
file_path = huggingface_hub.hf_hub_download(
    repo_id="sayakpaul/ucf101-subset",
    filename="UCF101_subset.tar.gz",
    repo_type="dataset"
)
# file_path는 다운로드한 압축파일이 존재하는 경로와 파일명이 string으로 저장되어있음.
with tarfile.open(file_path) as t:
     t.extractall("./data") # 여기에서 "."은 현재폴더라는 의미

In [69]:
mp.tree("./data")

└── UCF101_subset
    ├── test
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g03_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g23_c06.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g14_c01.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g16_c04.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g05_c02.avi
    │       └── ...
    │       └── v_BenchPress_g25_c06.avi
    ├── train
    │   ├── ApplyEyeMakeup
    │   │   ├── UCF101
    │   │   ├── v_ApplyEyeMakeup_g02_c03.avi
    │   │   └── ...
    │   │   └── v_ApplyEyeMakeup_g25_c07.avi
    │   ├── ApplyLipstick
    │   │   ├── UCF101
    │   │   ├── v_ApplyLipstick_g01_c02.avi
    │   │   └── ...
    │   │   └── v_ApplyLipstick_g24_c05.avi
    │   └── ...
    │   └── BenchPress
    │       ├── UCF101
    │       ├── v_BenchPress_g01_c05.avi
    │       └── ...
    │       └── v_BenchPress_g24_c

In [70]:
video_path = "./data/UCF101_subset/test/BenchPress/v_BenchPress_g05_c02.avi"
video = pytorchvideo.data.encoded_video.EncodedVideo.from_path(video_path).get_clip(0, float('inf'))['video']
video.shape

torch.Size([3, 67, 240, 320])

`(풀이)`

In [71]:
model3(
    #video.permute(1,0,2,3)[:16,:,:224,:224].unsqueeze(0)
    #video.permute(1,0,2,3)[:16,:,:224,:224].reshape(1,16,3,224,224)
    torch.stack([video.permute(1,0,2,3)[:16,:,:224,:224]])
)

ImageClassifierOutput(loss=None, logits=tensor([[-0.1375,  0.4270]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)